# Classificação de sentimentos do Twitter

## Autor

Samuel Nogueira Bacelar - 180130722

Github: [SamuelNoB](https://github.com/SamuelNoB)

## Objetivo

O objetivo desse artigo é treinar um modelo capaz de identificar os textos de comentários do Twitter e classificá-los de acordo com o sentimento como positiva ou negativa.

## Motivo

A fim de aplicar conhecimentos de NLP uma das formas de utilizá-lo é a partir da classificação textual. Além disso, os comentários e postagens (tweets) em redes sociais como o twitter podem necessitar de uma classificação prévia e simples de modo a facilitar a identificação de publicações nocivas e que podem vir a ferir as regras da plataforma.

## Instalação de depêndencias

In [1]:
! pip install transformers datasets evaluate

You should consider upgrading via the '/usr/src/python/samuel/fastaiOnCampus/.venv/bin/python -m pip install --upgrade pip' command.


Para realizar o deploy, irei utilizar o *notebook_login* do huggingface para acessar minha conta e poder salvar o modelo.

Dessa forma, será feita a instalação do pacote *huggingface_hub* e a utilização do login dele.

Ao executar a função *notebook_login()* um prompt aparece com um campo para adicionar um token de acesso disponibilizado pelo huggingface.

In [2]:
! pip install huggingface_hub

You should consider upgrading via the '/usr/src/python/samuel/fastaiOnCampus/.venv/bin/python -m pip install --upgrade pip' command.


In [1]:
from huggingface_hub import notebook_login

notebook_login()

## Preparando Dataset

A base de dados utilizada esta disponível no [kaggle](https://www.kaggle.com/datasets/kazanova/sentiment140?resource=download) e possui 1 CSV que contem 1.6 milhão de tweets do twitter. E no intuito de criar um dataset de validação, iremos repartir o dataset em 2 em uma proporção de 80/20. O dataset de treino possui 80% dos dados e o dataset de validação possui 20% dos dados.

In [3]:
import pandas as pd
import numpy as np

# DATASET
DATASET_COLUMNS = ["labels", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"

df = pd.read_csv('./data/training.1600000.processed.noemoticon.csv', encoding=DATASET_ENCODING, names=DATASET_COLUMNS)




O conjunto de dados tem 6 campos: 
- `label`: O sentimento do tweet (0 = negative, 2 = neutral, 4 = positive)
- `ids`: O ID do tweet ( 2087)
- `date`: A data do tweet (Sat May 16 23:58:44 UTC 2009)
- `flag`: A Query (lyx). Se não há query, a seguinte flag está presente NO_QUERY.
- `user`: O usuário que twettou (robotickilldozr)
- `text`: o texto do tweet.

In [4]:
df

,labels,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [5]:
df = df.drop(columns=['date', 'flag', 'user', 'ids'])
df['labels'] = df['labels'].apply(lambda number: number/4)
df_train, df_valid = np.array_split(df, [int(len(df) * 0.8)])

In [6]:
df_train

,labels,text
0,0.0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0.0,is upset that he can't update his Facebook by ...
2,0.0,@Kenichan I dived many times for the ball. Man...
3,0.0,my whole body feels itchy and like its on fire
4,0.0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1279995,1.0,@MariaJEchelon -- 30 Seconds to Mars- The Fan...
1279996,1.0,@swaffette was a fab party And the new stuff ...
1279997,1.0,making the kittens talk in lolspeak.
1279998,1.0,still aching after yestdy's board sesh.. off t...


In [7]:
df_train.describe(include='object')

,text
count,1280000
unique,1265211
top,isPlayer Has Died! Sorry
freq,210


Após rodar o comando `describe()` fomos capazes de obter alguns insights interessantes. Um total de 659.775 usuários diferentes realizaram 1.6 milhão de tweets. O usuário 'list_dog' foi o que mais realizou tweets, com um total de 549 publicações.

Iremos então transformar o dataframe em dataset utilizando a lib *datasets* para que ele seja compatível com nosso tokenizer

In [8]:
from datasets import Dataset, DatasetDict

ds_train = Dataset.from_pandas(df_train)
ds_train

Dataset({
    features: ['labels', 'text'],
    num_rows: 1280000
})

## Pré-processamento do texto (Tokenizando e Numeralização)

O processo de tokenização é fundamental no NLP. Ele se resume em converter o texto em lista de palavras (podendo ser em caracteres,ou substrings, dependendo da granularidade do modelo utilizado).

Após realizar a tokenização, será obtido um vocabulário de palavras únicas presentes no dataset. Como um modelo de deep learning espera números como inputs essas palavras(tokens) são convertidas em números.

O modelo utilizado nesse artigo será o *cardiffnlp/twitter-roberta-base-sentiment*.

O Twitter-roBERTa-base  base model é uma versão do oriunda do *roBERTa-base model*, treinado com 58 milhoes de tweets. Ele não diferencia caixa alta e caixa baixa, é mais rápido e menor que o BERT.

In [9]:
from transformers import AutoTokenizer

model_dbu = "cardiffnlp/twitter-roberta-base-sentiment"

toks = AutoTokenizer.from_pretrained(model_dbu)

Função para tokenizar o texto e truncar sequências, se necessário, para que não sejam maiores que o input aceito pelo modelo roBERTa.

In [10]:
def tokenizer_function(examples):
    return toks(examples["text"], truncation=True)

Aplicando a função a todos os elementos do dataset com a função map.

In [11]:
tokenized_tweets = ds_train.map(tokenizer_function, batched=True)

Map:   0%|          | 0/1280000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


### Exemplo de tokenização e Numeralização

Segue um exemplo de como o *tokenize()* separa o texto em "tokens" e como cada um recebe um número no vocabulário.

In [12]:
toks.tokenize("this week is not going as i had hoped ")

['this', 'Ġweek', 'Ġis', 'Ġnot', 'Ġgoing', 'Ġas', 'Ġi', 'Ġhad', 'Ġhoped', 'Ġ']

In [13]:
toks.vocab['week']

3583

Agora observe um dos textos "tokenizados" do dataset utilizado

In [14]:
row = tokenized_tweets[23]
row['text'], row['input_ids']

('this week is not going as i had hoped ',
 [0, 9226, 186, 16, 45, 164, 25, 939, 56, 5207, 1437, 2])

Agora será criado um lote de exemplos usando o *DataCollatorWithPadding()*, esse lote será utilizado posteriormente como um dos parametrôs do treinamento do modelo.

In [15]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=toks)

### Separando o dataset em teste e treino e validação

Para realização do treinamento e testar o modelo após o dataset será dividido em 2 partes, uma contendo 20% dos dados (dados de teste) e a outra com os 80% restantes.

Pra isso foi utilizado o método *train_test_split()*.

In [16]:
dds = tokenized_tweets.train_test_split(0.20, seed=42)

dds

DatasetDict({
    train: Dataset({
        features: ['labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1024000
    })
    test: Dataset({
        features: ['labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 256000
    })
})

In [17]:
dds = dds.remove_columns('text')
dds

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1024000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 256000
    })
})

Realizando a limpeza das labels do dataframe de validação, e convertendo em dataset para utilização posterior ao treinamento do modelo.

Como pode ser visto o dataset de validação é criado a partir de outro dataframe com dados diferentes do dataframe que gerou o dataset de treino e teste

In [18]:
df_valid.drop(['labels'], axis=1, inplace=True)
ds_valid = Dataset.from_pandas(df_valid)

Realizar a separação dos datasets tem um motivo importantissimo.

Isso evita que ocorra o **overfitting** e o **underfitting**

Uma breve explicação:

Um modelo é basicamente um sistema para mapear entradas para saídas. Em suma, um modelo aprende relacionamento entre as entradas (features) e as saídas (labels) a partir de um dataset de treinamento. Depois de treinado o modelo deve ser capaz de receber apenas as entradas (dataset de testes) e ele faz previsões sobre as saídas a partir do que aprendeu no treinamento.

O Modelo mais simples é uma regressão linear e para aumentar a complexidade desse modelo basta aumentar o grau polinomial.

$$
   y = \beta_0x + \beta_1 x² = + \beta_2 x² + ... + \beta_n x^n + \epsilon
$$

![](https://miro.medium.com/max/720/1*pjIp920-MZdS_3fLVhf-Dw.webp)

O grau do polinomial é a representação da flexibilida do modelo. E a partir disso podemos entender o que é o undefit e o overfit de um modelo.

Um modelo underfit será menos flexível, o que o impede de contabilizar os dados como visto na imagem a seguir.


|||
|--|--|
|![](https://miro.medium.com/max/640/1*kZfqaD6hl9iYGYXkMwV-JA.webp)| ![](https://miro.medium.com/max/640/1*2RXJ2O-_c2ukaq5p-WQ9tQ.webp)

A função do modelo na cor laranja, na imagem à esquerda, está acima da função real e do conjunto de dados de treinamento. À direita está a previsão dada pelo modelo. Basicamente o modelo ignora o conjunto de treinamento, pois o modelo underfit tem uma **baixa varância**(depende pouco dos dados de treinamento) e **alto viés** (faz uma forte suposição sobre o comportamento dos dados). Quando o modelo tenta fazer previsões o alto viés o faz ter previsões com alta imprecissão.

|||
|--|--|
|![](https://miro.medium.com/max/640/1*Di7rY6ALXtkhlmlcKRSCoA.webp)| ![](https://miro.medium.com/max/640/1*QzA45ATjeEbwv5f1G99GnQ.webp)

Quando o grau do modelo é aumentado muito, o modelo consegue alcançar todas alterações dos dados de treinamento, essa alta flexibilidade  faz com que o modelo tenha uma ótima precisão nos dados de treinamento, mas os dados possuem ruídos e essa **alta flexibilidade** faz com que o modelo se ajuste a eles. Assim o modelo irá possuir uma **alta variância**, ele basicamente **memoriza os dados de treinamento** e não "aprende" como é o desejado.

## Avaliação

Para avaliar o desempenho do modelo é preciso ter uma métrica. 

Para simplificar a obtenção de um método que possa fazer essa avaliação do modelo, utilizaremos a biblioteca evaluate que possui duzias de métodos de avaliação pra diferentes domínios além do NLP.

A métrica que decide por utilizar foi a métrica de *accuracy*

In [19]:
import evaluate

accuracy = evaluate.load("accuracy")

Essa função utiliza da previsão feita e da label para calcular a métrica utilizada (acurácia).

In [20]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

## Treinamento

Map para converter as labes em palavras significantes e o contrário

In [21]:
id_label = {0: "NEGATIVE", 0.5: "NEUTRAL", 1: 'POSITIVE'}
label_id = {"NEGATIVE": 0, "NEUTRAL": 0.5, 'POSITIVE': 1}

Carregando o modelo pré treinado roBERTa com o número de labels e o mapeamento delas

In [22]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    model_dbu, 
    num_labels=3, 

    id2label=id_label, 
    label2id=label_id
)

In [27]:
import torch
torch.unsqueeze(['label', 'label2', 'label3'], 1)

TypeError: unsqueeze(): argument 'input' (position 1) must be Tensor, not list

Instalando as depenências de git para ser possível fazer o push do arquivo do modelo para o HugginFace

In [23]:
! sudo apt-get install git-lfs
! git lfs install

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Lendo listas de pacotes... Pronto
Construindo árvore de dependências... Pronto
Lendo informação de estado... Pronto        
git-lfs is already the newest version (3.3.0).
0 pacotes atualizados, 0 pacotes novos instalados, 0 a serem removidos e 115 não atualizados.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Updated Git hooks.
Git LFS initialized.


Definindo os **argumentos do treinamento com *TrainingArguments()***
- `output_dir`: Onde salvar o modelo;
- `learning_rate`: taxa de aprendizagem inicial;
- `per_device_train_batch_size`: o tamanho da amostra de treino por GPU/TPU;
- `per_device_eval_batch_size`: o tamanho da amostra de avaliação por GPU/TPU;
- `num_train_epochs`: Número de épocas para o treinamento;
- `weight_decay`: A taxa de decaimento do peso a ser aplicada as camadas;
- `evaluation_strategy`: A estrátegia de avaliação adotada no treinamento;
- `save_strategy`: A estratégia do ponto de salvamento a ser adotada durante o treinamento;
- `load_best_model_at_end`: Carregar ou não o melhor modelo encontrado durante o treinamento (Quando verdadeiro exige que os parâmetros "evaluation_strategy" e "save_strategy" sejam iguais);
- `push_to_hub`: "Push" do modelo para o Hub, quando ele for salvo (depende do "output_dir").

Conhecendo os parâmetros utilizados para a classe *transofrmers.Trainer*:
- `model`: O modelo a ser treinado, avaliado ou utilizado para previsão;
- `args`: Os argumentos para ajustar o treinamento;
- `train_dataset`: O conjunto de dados que será utilizado para o treinamento,
- `eval_dataset`: O conjunto de dados utilizado para avaliação;
- `tokenizer`: O tokenizador usado para pré-processar os dados;
- `data_collator`: A função utilizada para formar um lote a partir de uma lista de elementos de train_dataset ou eval_dataset;
- `compute_metrics`: a função que será utilizada paar computar as métricas na avaliaçã.

In [24]:



training_args = TrainingArguments(
    output_dir="classification_text_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
    no_cuda=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dds["train"],
    eval_dataset=dds["test"],
    tokenizer=toks,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    
)


/usr/src/python/samuel/fastaiOnCampus/nbs/classification_text_model is already a clone of https://huggingface.co/SamuelNog/classification_text_model. Make sure you pull the latest changes with `repo.git_pull()`.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [25]:
trainer.train()

/usr/src/python/samuel/fastaiOnCampus/.venv/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: Target size (torch.Size([16])) must be the same as input size (torch.Size([16, 3]))

O tempo de treinamento foi longo, apesar de utilizar apenas 2 épocas e realizar o treinamento em uma maquina do kaggle com o uso da GPU T4 X2, o treinamento do modelo demorou cerca de 30 minutos. 

Assim como configurado nos argumentos o modelo foi salvo em um [repositório](https://huggingface.co/lucasgbezerra/classification_text_model) com o nome especificado na conta logada no ínicio do artigo.

## Testando o modelo

Vamos verificar o que o modelo prevê a partir do dataset de testes e validação

In [ ]:
preds = trainer.predict(dds['test']).predictions.astype(float)
preds

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 8000
  Batch size = 32
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


array([[ 2.23709369, -2.29509497],
       [-1.78306019,  1.47043931],
       [-1.34510422,  0.93321747],
       ...,
       [-2.40362573,  1.9160639 ],
       [ 0.50562054, -0.47438526],
       [-2.07757139,  1.67809868]])

A previsão do dataset de testes retornou números menores que -1 e maiores que 1. Para melhorar o entendimento sobre os resultados e definir limites vamos utilizar o método clip do numpy para deixar estes números dentre o range 0 à 1, que nos permite visualizar como porcentagem o grau de positivadade ou negatividade das frases avaliadas.

In [ ]:
preds = np.clip(preds, 0, 1)
preds

array([[1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 0.93321747],
       ...,
       [0.        , 1.        ],
       [0.50562054, 0.        ],
       [0.        , 1.        ]])

Agora utilizando outro conjunto de dados, os dados de validaçaão

Primeiro eles serão tokenizados para então poder ocorrer a previsão

In [ ]:
toks_valid = ds_valid.map(tokenizer_function, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
preds2 = trainer.predict(toks_valid).predictions.astype(float)
preds2

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 5000
  Batch size = 32


array([[ 1.95127702, -1.99396503],
       [ 2.37554598, -2.5228641 ],
       [ 2.17845559, -2.19129372],
       ...,
       [-1.58796299,  1.19799149],
       [-2.55518889,  2.01890063],
       [-2.28080058,  1.7151109 ]])

Novamente os resultados serão normalizados entre 0 e 1

In [ ]:
preds2 = np.clip(preds2, 0, 1)
preds2

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

## Resultado do deploy

Para realizar o deploy utilizei o modelo salvo no [HuggingFace]().

E a partir dele gerei um Space com o seguinte app.py:


```python
import gradio as gr

description = "Classificador de sentimento (Positivo, Negativo) de textos de avaliações de filmes"
examples = [
#             lista de exemplos disponíveis para usar o modelo  
        ]

gr.Interface.load("models/lucasgbezerra/classification_text_model",  description=description, examples=examples).launch()
```

Para acessar o deploy basta [clicar aqui]()



## Conclusão

Aplicar um modelo pré treinado para NLP é um desafio maior do que o encontrado nos tópicos anteriores de Vision Computer.

A utilização das libs do HugginFace facilitaram bastante, principalmente a nível de entendimento.

A maior dificuldade, quando comparado a Vision Computer está na preparação dos dados. Dados visuais são muito mais simples de serem separados, e erros mais simples de serem notados no conjunto de dados, quando os dados passam a ser textuais a interpretação deles é mais complexa.

Outro ponto a se destacar é o tempo gasto no treinamento, no modelo NLP o tempo gasto em 2 épocas é muito alto, provavelmente muito disso seja devido a quantidade de dados utilizados, cerca de 40 mil textos.

Por último, vale destacar que quanto mais se aprofunda nos parâmetros para a construção do modelo, maior o grau de conhecimento se torna necessário. Aspectos como o underfitting e o overfitting se tornam básicos para entender como seu modelo se porta e como melhorá-lo. A métrica de avaliação se torna mais complexa, e pode ser de diversos tipos, cada uma com um foco e se encaixando melhor em determinado modelo.